# __Presents all data from the Frequency Response Magnetophosphene study__
## effect of ELF-MF (up to 300Hz) on magnetophosphene perception threshold

In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from os import listdir
import re

In [49]:
plt.rcParams['figure.figsize'] = 10, 10
plt.rcParams['font.family']='mono'
plt.rcParams['font.size']=15

sns.set_style("whitegrid")

In [41]:
%matplotlib auto

Using matplotlib backend: MacOSX


## considering only subject # 13 to # 60
* 1 to 12 do not have 5Hz to 15Hz

In [42]:
folder_res = '../DATA/ResTables/'
list_files = [x for x in listdir(folder_res) if x.endswith('.csv')]

All_thr = pd.DataFrame()
Bad_ID = [13, 28, 39, 41, 50, 52, 54, 60]
# Bad_ID = []

for file_name in list_files:
    _id = int(''.join(re.findall('[0-9]',file_name)))
    if _id in Bad_ID:
        continue
    data = pd.read_csv(folder_res+file_name)
    All_thr = All_thr.append(data)

# dB/dt threshold
All_thr['dBdt'] = All_thr['n_Threshold'] * All_thr['Frequency']

## Extract percentage of perception for each frequency

In [43]:
frequencies = All_thr.Frequency.unique()
frequencies.sort()

In [44]:
distr_perceived = pd.Series(index = frequencies, dtype=float)

for fr in frequencies:
    df = All_thr.query(f'Frequency=={fr}')['Perceived']
    percentage = df.sum()/len(df)

    
    distr_perceived[fr] = percentage

### Perception rate over frequency

In [45]:
fig, ax  = plt.subplots()
pl1 = ax.plot(distr_perceived.index, distr_perceived, color='blue', lw='1', marker='.')
ax.fill_between(distr_perceived.index, 0, distr_perceived, alpha=.3)

ax.plot(distr_perceived[distr_perceived > 0.8], 'k.', ms=15)
# ax.plot(distr_perceived[distr_perceived > 0.95], 'r.')

subfreq=[0,10,20,30,40,50,60,70,80,90,100,150,200,250,300]
plt.xticks(subfreq)
plt.xlabel('MF Frequency (Hz)', {'fontweight':'bold'})
plt.ylabel('Perception rate', {'fontweight':'bold'})

Text(0, 0.5, 'Perception rate')

## raw representation of threshold

In [46]:
plt.figure()
sns.stripplot(x="Frequency", y="Threshold", data=All_thr, color='black')
sns.stripplot(x="Frequency", y="n_Threshold", data=All_thr, color='grey')
plt.ylabel('Threshold (mT)', {'fontweight':'bold'})
plt.ylabel('Frequency (Hz)', {'fontweight':'bold'})

Text(0, 0.5, 'Frequency (Hz)')

---
# __Regression models__
> to express threshold value as a function of frequency

We chose only the frequncies for which we have over 80% perception

### __Using StatsModels for statistics__
https://ostwalprasad.github.io/machine-learning/Polynomial-Regression-using-statsmodel.html

### __Using sklearn linear_model - polynomialFeatures__
(can be done with numpy poly1d)

### __Using scipy for curve optimization with exponential model__

In [47]:
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

from scipy.optimize import curve_fit


1. preparing the data for modeling

In [57]:
idxFreq = distr_perceived[distr_perceived >0.8].index.values.tolist()
data = All_thr.query(f'Frequency in {idxFreq}')[['Frequency','n_Threshold','dBdt']]

#  remove nan
data.dropna(subset=['n_Threshold'], inplace=True)

Freq = data.Frequency[:, np.newaxis]
Thres = data.n_Threshold[:, np.newaxis]
dBdT = data.dBdt[:, np.newaxis]

response = Thres
nFreq = np.arange(5,100)[:, np.newaxis]

2. Polynomial model $y = ax^{2} + bx + c$

In [13]:
polynomial_features = PolynomialFeatures(degree=2)
x = polynomial_features.fit_transform(Freq)
new_x = polynomial_features.fit_transform(nFreq)

3. Simple model $y = ax^{2} + c$

In [58]:
x = sm.add_constant(Freq**2)
new_x = sm.add_constant(nFreq**2)

4. Exponential model $y = ae^{-bx} + c$

In [51]:
def func(x,a,b,c):
    return a * np.exp(-b * x) + c

coef_opt, cov = curve_fit(func, data['Frequency'], data['dBdt'])

/Users/sebastienvillard/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


### __Results__

In [59]:
model = sm.OLS(response, x).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.101
Model:                            OLS   Adj. R-squared:                  0.100
Method:                 Least Squares   F-statistic:                     119.1
Date:                Thu, 30 Apr 2020   Prob (F-statistic):           2.31e-26
Time:                        13:45:13   Log-Likelihood:                -4121.5
No. Observations:                1063   AIC:                             8247.
Df Residuals:                    1061   BIC:                             8257.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         26.8507      0.575     46.726      0.000      25.723      27.978
x1             0.0023      0.000     10.915      0.000       0.002       0.003
==============================================================================
Omnibus:                      121.440   Durbin-Watson:                   1.284
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              166.532
Skew:                           0.873   Prob(JB):                     6.89e-37
Kurtosis:                       3.844   Cond. No.                     4.41e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.41e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### __Predictions__

__compute predictions with Confidence intervals & Predcition intervals__

In [60]:
predictions = model.get_prediction(new_x)
alpha = 0.50
conf_95 = predictions.summary_frame()
conf_01 = predictions.summary_frame(alpha=alpha)
display(conf_01)

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,26.907662,0.570573,26.522685,27.292640,19.006736,34.808589
1,26.932745,0.568788,26.548972,27.316518,19.031877,34.833613
2,26.962388,0.566683,26.580035,27.344741,19.061589,34.863187
3,26.996592,0.564261,26.615873,27.377310,19.095872,34.897312
4,27.035356,0.561524,26.656483,27.414228,19.134724,34.935987
...,...,...,...,...,...,...
90,47.429767,1.480514,46.430832,48.428701,39.475252,55.384281
91,47.865291,1.519256,46.840217,48.890365,39.907452,55.823131
92,48.305376,1.558462,47.253849,49.356904,40.344086,56.266666
93,48.750022,1.598131,47.671729,49.828315,40.785153,56.714891


In [78]:
fig, ax1 = plt.subplots()
nFreq = nFreq.ravel()
plt.plot(nFreq,conf_95['mean'],'b', linewidth=3, label="Mean")
plt.plot(nFreq,conf_95['mean_ci_lower'],'cyan')
plt.plot(nFreq,conf_95['mean_ci_upper'],'cyan')
plt.fill_between(nFreq, conf_95.mean_ci_lower, conf_95.mean_ci_upper, alpha=.3)

plt.plot(nFreq,conf_01.obs_ci_upper,':',label=f"{(1-alpha)*100}%")
plt.plot(nFreq,conf_95.obs_ci_upper,':',label="87.5%")
plt.plot(nFreq,conf_95.obs_ci_lower,':',label="2.5%")
plt.plot(nFreq,conf_01.obs_ci_lower,'--',label=f"{alpha*100}%",linewidth=2)

# plt.plot(All_thr.Frequency, All_thr.n_Threshold, 'k.')
plt.plot(Freq, response, 'r.', ms=2)

plt.xlabel('MF Frequency (Hz)')
plt.ylabel('Threshold (mT)')

plt.legend()

# plot perception rate
if False:
    ax2 = ax1.twinx()
    ax2.plot(distr_perceived.index, distr_perceived, color='blue', lw='0.5')
    ax2.fill_between(distr_perceived.index, 0, distr_perceived, alpha=.1)

In [ ]:
# fig.savefig('../ThresholdModels.png',dpi=300)

In [ ]:
# data[data.Threshold > 70].to_csv('HighThreshold.csv')